#### World Bank Projects Dataset

In this notebook, I'll find the 10 countries with the most projects, find the top 10 major project themes, and clean the column associated with the names of the major themes.


In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

#load json into Pandas dataframe
json_df = pd.read_json('data/world_bank_projects.json')
json_df.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


In [2]:
#check if the column containing country values has any NAN values
print(json_df['countryshortname'].isnull().values.all())

False


In [3]:
#no Nan, it is a reliable source to find the counts

most_projects = json_df['countryshortname'].value_counts()
print(most_projects.head(10))    

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Bangladesh            12
Nepal                 12
Morocco               12
Africa                11
Mozambique            11
Name: countryshortname, dtype: int64


In [4]:
#Each row of the 'mjtheme_namecode' column consists of a list of dictionaries
#containing keys 'code' and 'name' and values associated with them.  First,
#I will extract only that particular column, initialize a dict to hold the 
#theme name as key, and the counts as the value.  A double for loop counts all
#instances of a theme.

theme_column = json_df['mjtheme_namecode']
counts = {}

for row in theme_column:
    for dict in row:
        if dict['name'] in counts:
            counts[dict['name']] += 1
        else:
            counts[dict['name']] = 0

#It would be nice to list the dictionary in order          
#sequence for sorting dict found at:     
#https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value

import operator
sorted_counts = sorted(counts.items(), key = operator.itemgetter(1), reverse = True)

#print top 10
for key, value in sorted_counts[:10]:
    print(key, ': ', value) 

Environment and natural resources management :  222
Rural development :  201
Human development :  196
Public sector governance :  183
Social protection and risk management :  157
Financial and private sector development :  129
 :  121
Social dev/gender/inclusion :  118
Trade and integration :  71
Urban development :  46


In [5]:
#There are 121 instances where a row contained a dictionary with a value of ''
#corresponding to the theme name.  Before fixing, I'll make a dict that matches
#the correct code to the correct name, so we can fill all empty values properly.

theme_key = {}

for row in theme_column:
    for dict in row:
        if (dict['code'] not in theme_key) or (theme_key[dict['code']] == ''):            
            theme_key[dict['code']] = dict['name']

print(theme_key)

{'8': 'Human development', '11': 'Environment and natural resources management', '1': 'Economic management', '6': 'Social protection and risk management', '5': 'Trade and integration', '2': 'Public sector governance', '7': 'Social dev/gender/inclusion', '4': 'Financial and private sector development', '10': 'Rural development', '9': 'Urban development', '3': 'Rule of law'}


In [6]:
#Use the theme_key dict to replace each empty name with the appropriate text for the code value.

for row in theme_column:
    for dict in row:
        if dict['name'] == '':
            dict['name'] = theme_key[dict['code']]


In [7]:
#count again, but now there won't be any blank values corresponding to 'name'

true_counts = {}

for row in theme_column:
    for dict in row:
        if dict['name'] in true_counts:
            true_counts[dict['name']] += 1
        else:
            true_counts[dict['name']] = 0

            
#sort the dictionary
sorted_true_counts = sorted(true_counts.items(), key = operator.itemgetter(1), reverse = True)

#print top 10
for key, value in sorted_true_counts[:10]:
    print(key, ': ', value) 

Environment and natural resources management :  249
Rural development :  215
Human development :  209
Public sector governance :  198
Social protection and risk management :  167
Financial and private sector development :  145
Social dev/gender/inclusion :  129
Trade and integration :  76
Urban development :  49
Economic management :  37


In [8]:
#Make a copy of the dataframe and replace the column that has been cleaned.
final_df = json_df.copy()
final_df['mjtheme_namecode'] = theme_column
final_df.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17c783'},2014,October,2013-10-31T00:00:00Z,MIN. OF PLANNING AND INT'L COOPERATION,NaN,Republic of Yemen!$!RY,RY,Republic of Yemen,"Yemen, Republic of",...,JB,IBRD,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
4,{'$oid': '52b213b38594d8a2be17c784'},2014,October,2013-10-31T00:00:00Z,MINISTRY OF FINANCE,2019-04-30T00:00:00Z,Kingdom of Lesotho!$!LS,LS,Kingdom of Lesotho,Lesotho,...,"FH,YW,YZ",IBRD,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...
